In [ ]:
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
start_time = time.time()

from sklearn.preprocessing import RobustScaler
from functools import reduce
from sklearn.preprocessing import StandardScaler
from keras import backend as K
import tensorflow as tf

import random
import numpy as np
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)

Windows: 1,2,3,6,12,22,45,90

In [ ]:
numberOfWindows = 90
Restriction = '90'
ep = 125
numberOfFeatures = 30

los_3

In [ ]:
import pandas as pd

In [ ]:
FeatureSetB2 = pd.DataFrame()
FeatureSetB3 = pd.DataFrame()
FeatureSetB4 = pd.DataFrame()

In [ ]:
FeatureSetB1 = pd.read_csv('DI/DATA_Base.csv') #Split
FeatureSetB2 = pd.read_csv(f'DI/DATA_{numberOfWindows}.csv')#Train

In [ ]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
FeatureSetB1.drop("Unnamed: 0",axis=1,inplace = True)
FeatureSetB2.drop("Unnamed: 0",axis=1,inplace = True)

FeatureSetB1 = FeatureSetB1.fillna(0)
FeatureSetB2 = FeatureSetB2.fillna(0)

FeatureMortality2 = pd.read_csv('DI/Sleep_Tbl.csv') #Labels
FeatureMortality2.drop("Unnamed: 0",axis=1,inplace = True)
FeatureMortality = pd.DataFrame()
FeatureMortality['subject_id'] = FeatureMortality2['ClientId']
FeatureMortality['Flags'] = FeatureMortality2['Flags']

FeatureSetB1.sort_values(by=['subject_id'], ascending=True,inplace=True)

FeatureMortality.sort_values(by=['subject_id'], ascending=True,inplace=True)

In [ ]:
subject_id = pd.DataFrame()
subject_id = FeatureSetB1.subject_id
subject_id.drop_duplicates(keep = 'first', inplace = True)
subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(subject_id)

In [ ]:
X = FeatureMortality['subject_id'].isin(subject_id)
FeatureMortality = FeatureMortality[X]
FeatureMortality.sort_values(by=['subject_id'], ascending=True,inplace=True) 

In [ ]:
subject_id = pd.DataFrame()
subject_id = FeatureMortality.subject_id
subject_id.drop_duplicates(keep = 'first', inplace = True)
subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(subject_id)
X = FeatureSetB1['subject_id'].isin(subject_id)
FeatureSetB1 = FeatureSetB1[X]

In [ ]:
FeatureMortality.Flags[FeatureMortality.Flags == 1] = 0
FeatureMortality.Flags[FeatureMortality.Flags == 2] = 1      

In [ ]:
FeatureMortality['Flags'].value_counts()         #Chronic is 1 

In [ ]:
Labels = pd.DataFrame()
Labels['subject_id'] = FeatureMortality['subject_id']
Labels['Flags'] = FeatureMortality['Flags']

In [ ]:
y = Labels

In [ ]:
X = FeatureSetB1
X_1 = [X,y]

X_2 = reduce(lambda  left,right: pd.merge(left,right,on=['subject_id'],how='outer'), X_1)
X_2.sort_values(by=['subject_id'], ascending=True,inplace=True)

In [ ]:
X_2 = X_2.dropna(subset=['Flags'])

In [ ]:
subject_id = pd.DataFrame()
subject_id = X_2.subject_id
subject_id.drop_duplicates(keep = 'first', inplace = True)
subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(subject_id)

In [ ]:
X = FeatureMortality['subject_id'].isin(subject_id)
FeatureMortality = FeatureMortality.reset_index(drop=True)
FeatureMortality = FeatureMortality[X]
FeatureMortality.sort_values(by=['subject_id'], ascending=True,inplace=True) 

In [ ]:
X_age = X_2['age'].mean()
X_emp = X_2['emp'].mean()
X_emc = X_2['emc'].mean()
X_sleep = X_2['sleep'].mean()
X_ssAgg = X_2['ssAgg'].mean()

In [ ]:
X_2.loc[(X_2.age < X_age),  'age_Group'] = 'Young'
X_2.loc[(X_2.age >= X_age),  'age_Group'] = 'Old'

X_2.loc[(X_2.emp < X_emp),  'emp_Group'] = 'Low'
X_2.loc[(X_2.emp >= X_emp),  'emp_Group'] = 'High'

X_2.loc[(X_2.emc < X_emc),  'emc_Group'] = 'Low'
X_2.loc[(X_2.emc >= X_emc),  'emc_Group'] = 'High'

X_2.loc[(X_2.sleep < X_sleep),  'sleep_Group'] = 'Low'
X_2.loc[(X_2.sleep >= X_sleep),  'sleep_Group'] = 'High'

In [ ]:
X_2['Flags'].value_counts()

In [ ]:
X_2['age_Group'].value_counts()

In [ ]:
X_2['emp_Group'].value_counts()

In [ ]:
X_2['emc_Group'].value_counts()

In [ ]:
X_2['sleep_Group'].value_counts()

In [ ]:
X = FeatureSetB1

X_train1, X_test = train_test_split(X_2, test_size=0.25, stratify=X_2[['Flags']], shuffle=True,random_state=42)#,'age_Group','emp_Group','emc_Group','sleep_Group']]#,'AIDS','MetaStatic','Hematologic'#,'GCSEyes']]

In [ ]:
X_train1['Flags'].value_counts()

In [ ]:
X_train1['sleep_Group'].value_counts()

In [ ]:
X_train1['emc_Group'].value_counts()

In [ ]:
X_train1['emp_Group'].value_counts()

In [ ]:
X_train1['age_Group'].value_counts()

In [ ]:
X_train, X_val = train_test_split(X_train1, test_size=0.2, random_state=42, stratify=X_train1[['Flags']], shuffle=True)#,'age_Group','emp_Group','emc_Group','sleep_Group']], shuffle=True)#, shuffle=True)#,'GCSEyes']],

In [ ]:
X_test['Flags'].value_counts()

In [ ]:
X_val['Flags'].value_counts()

In [ ]:
X_train['Flags'].value_counts()

In [ ]:
Subject_Test = pd.DataFrame()
Subject_Test = X_test.subject_id

Subject_Train = pd.DataFrame()
Subject_Train = X_train.subject_id

Subject_Val = pd.DataFrame()
Subject_Val = X_val.subject_id

In [ ]:
X = FeatureMortality['subject_id'].isin(Subject_Train)
y_train = FeatureMortality[X]

X = FeatureMortality['subject_id'].isin(Subject_Test)
y_test = FeatureMortality[X]

X = FeatureMortality['subject_id'].isin(Subject_Val)
y_val = FeatureMortality[X]


In [ ]:
y_test.sort_values(by=['subject_id'], ascending=True,inplace=True) 

In [ ]:
X = FeatureSetB2['subject_id'].isin(Subject_Train)
X_train = FeatureSetB2[X]


X = FeatureSetB2['subject_id'].isin(Subject_Test)
X_test = FeatureSetB2[X]


X = FeatureSetB2['subject_id'].isin(Subject_Val)
X_val = FeatureSetB2[X]

In [ ]:
Train_shape = X_train.shape
Test_shape = X_test.shape
Val_shape = X_val.shape

In [ ]:

X_train.drop("subject_id",axis=1,inplace = True)


X_test.drop("subject_id",axis=1,inplace = True)


X_val.drop("subject_id",axis=1,inplace = True)

In [ ]:
X_train = X_train.to_numpy()

X_test = X_test.to_numpy()

X_val = X_val.to_numpy()

In [ ]:
scaler1 = StandardScaler().fit(X_train)
scaler2 = StandardScaler().fit(X_test)
scaler3 = StandardScaler().fit(X_val)

X_train = scaler1.transform(X_train)
X_test = scaler2.transform(X_test)
X_val = scaler3.transform(X_val)

In [ ]:
X_train = X_train.reshape(int(Train_shape[0]/numberOfWindows),numberOfWindows,numberOfFeatures)
X_test = X_test.reshape(int(Test_shape[0]/numberOfWindows),numberOfWindows,numberOfFeatures)
X_val = X_val.reshape(int(Val_shape[0]/numberOfWindows),numberOfWindows,numberOfFeatures)

In [ ]:
X_train4 = X_train

X_test4 = X_test

X_val4 = X_val

In [ ]:
y_train = y_train['Flags']
y_train = y_train.to_numpy()
y_train = y_train.flatten()

y_test = y_test['Flags']
y_test = y_test.to_numpy()
y_test = y_test.flatten()

y_val = y_val['Flags']
y_val = y_val.to_numpy()
y_val = y_val.flatten()

Testing LR

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten, Activation
from sklearn import metrics
from keras.layers import LSTM
import tensorflow as tf
import sys
mod=sys.modules[__name__]

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 45, mode='min',restore_best_weights=True)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score



acc_score = []
re_score = []
pre_score = []
history_1 = []
f_score = []
acc_macro_score = []

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

In [ ]:
numberOfFeatures2 = X_test4.shape[1]

In [ ]:
minority_size = sum(y_train.astype(int) == 1)
majority_size = int(minority_size * 5)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.layers import GRU, Dense, Dropout




for x in range(40):
    model = Sequential()
    
    model.add(GRU(256, return_sequences=True, input_shape=(numberOfWindows ,numberOfFeatures)))  # Adjust the input shape depending on your data's time steps and features
    model.add(Dropout(0.2))
    
    model.add(GRU(128, return_sequences=False))
    model.add(Dropout(0.2))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid'))

    opt = tf.keras.optimizers.legacy.Adam()  # If you need specific learning rate or other parameters, define them here.
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = dict(zip(np.unique(y_train), class_weights))

    history = model.fit(X_train4, y_train, epochs=ep, batch_size=1024, validation_data=(X_val4, y_val), 
                        callbacks=[stop_early], shuffle=True, class_weight=class_weights)
    
    y_preds = model.predict(X_test4)
    y_preds = y_preds.flatten()
    y_pred = [1 if pred >= 0.5 else 0 for pred in y_preds]  # a simpler way to apply the threshold

    y_actu = pd.Series(y_test)
    y_pred = pd.Series(y_pred)

    sensitivity = recall_score(y_actu, y_pred, average='macro')
    precision = precision_score(y_actu, y_pred, average='macro')
    f1_value = f1_score(y_actu, y_pred, average='macro')
    accuracy = accuracy_score(y_actu, y_pred)

    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)

    matrix = confusion_matrix(y_actu, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix=matrix).plot()  # This will plot the confusion matrix. Use plt.show() if you're running this in a script.
    Accuracies = matrix.diagonal() / matrix.sum(axis=1)
    accuracy_macro = np.mean(Accuracies)  # assuming you have 2 classes, this calculates the mean
    acc_macro_score.append(accuracy_macro)

In [ ]:
k = 40
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_acc_macro_score = sum(acc_macro_score)/k

accuracy_macro = avg_acc_macro_score
sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score

In [ ]:
import math
def calculate_standard_deviation(numbers):
    n = len(numbers)
    mean = sum(numbers) / n
    variance = sum((x - mean) ** 2 for x in numbers) / n
    std_dev = math.sqrt(variance)
    return std_dev

std_acc = calculate_standard_deviation(acc_score)
std_acc_macro = calculate_standard_deviation(acc_macro_score)
std_recall = calculate_standard_deviation(re_score)
std_precision = calculate_standard_deviation(pre_score)
std_f1_score = calculate_standard_deviation(f_score)

In [ ]:
sensitivity_min = min(re_score)
sensitivity_max = max(re_score)

precision_min = min(pre_score)
precision_max = max(pre_score)

accuracy_min = min(acc_score)
accuracy_max = max(acc_score)

accuracy_macro_min = min(acc_macro_score)
accuracy_macro_max = max(acc_macro_score)

f1_score_min = min(f_score)
f1_score_max = max(f_score)

In [ ]:
print('Avg accuracy : {}'.format(avg_acc_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('Avg accuracy_macro : {}'.format(avg_acc_macro_score))

In [ ]:
for history in history_1:
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
for history in history_1:
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Validation F1')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
import csv
import os.path
from datetime import datetime
end_time = time.time()
runtime = end_time - start_time

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','accuracy_macro','NumberOfWindows','Epochs','Run_Time','Restriction','Acc_Lesser','Acc_Greater']
dict_data = [{'model-type':'ANN', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'accuracy_macro': accuracy_macro,'NumberOfWindows':numberOfWindows,"Epochs":ep,'Run_Time':runtime,'Restriction' : Restriction,'Acc_Lesser':Less10D,'Acc_Greater':Greater10D}]
metric_file = "Results/Episodic_ANN_Max_FR.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")